In [ ]:
import cv2
import time
import math
import numpy as np
import os
from imutils import perspective
import operator
import imutils
import copy
import random
from scipy.spatial import distance as dist
import mrcnn.config
import mrcnn.utils
from mrcnn.model import MaskRCNN
import os
import random 


In [ ]:
# Configuration that will be used by the Mask-RCNN library
class MaskRCNNConfig(mrcnn.config.Config):
    #NAME = "coco_pretrained_model_config"
    NAME = "nam,e"
    IMAGES_PER_GPU = 1
    GPU_COUNT = 1
    NUM_CLASSES = 1 + 1  
    DETECTION_MIN_CONFIDENCE = 0.9

In [ ]:
# Filter a list of Mask R-CNN detection results to get only the detected picsf 
def get_pic_boxes(boxes, class_ids):
    pic_boxes = []

    for i, box in enumerate(boxes):
        # If the detected object isn't a car, skip it
        if class_ids[i] in [1]:
            pic_boxes.append(box)

    return np.array(pic_boxes)


In [ ]:
# Root directory of the project
ROOT_DIR ="./"

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
MODEL_PATH = os.path.join(ROOT_DIR, "frame.h5")

# Create a Mask-RCNN model in inference mode
model = MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=MaskRCNNConfig())

print("root",ROOT_DIR)
print("model",MODEL_PATH)


In [ ]:
model.load_weights(MODEL_PATH, by_name=True)

# Location of parking spaces
parked_car_boxes = None

In [ ]:
path = ".\Frames_data"
files=os.listdir(path)
d=random.choice(files)
print(d)
path = os.path.join(path,d)
#img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
img = cv2.imread(path,-1) #  -1 = unchanged
img = cv2.resize(img, (900, 600))

if d[-3:]=="png":
    trans_mask = img[:,:,3] == 0
    img[trans_mask] = [255, 255, 255, 255]
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

cv2.imshow("Original Image",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

print(type(img),"size:",img.shape)
print("going in model shape:",img.shape)
results = model.detect([img], verbose=1)


r = results[0]
pic_boxes = get_pic_boxes(r['rois'], r['class_ids'])
print("total pics:",len(pic_boxes))
for box in pic_boxes:
    y1, x1, y2, x2 = box
    b = x1,y1,x2,y2
    #img = cv2.rectangle(img, (x1, y1), (x2, y2), (255,0 , 255),2)
    crop_img = img[y1:y2, x1:x2]
    cv2.imshow("Original Image",crop_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()